**第4章 Bonds**

  - `bondYield`メソッドはver1.39で仕様変更
    - myABBRモジュールで**債券価格クラス**を戻す<b>関数cP(...)</b>を設定 (cPはclean priceの略)
    - 下セル(図4.2)13行目はこの関数により、97.0を`cP(97.0)`へ修正
    - `zSpread`メソッド(図4.13)も同じ理由で97.0を`cP(97.0)`へ修正

In [1]:
from myABBR import * ; import myUTIL as mu
# 0.初期値設定
tradeDT,       settDS,     effDT,           matDT,     faceAMT,  cpnLST      =\
jDT(2022,8,19),  2,   jDT(2022,7,28), jDT(2025,7,28),  100.0,  [0.00370] 

settleDT = calJP.advance(tradeDT, settDS, DD); setEvDT(tradeDT)

# 1. スケジュールオブジェクトと債券オブジェクトの作成
bondSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, unADJ,unADJ, dtGENb, EoMf)
bondOBJ = ql.FixedRateBond(settDS, faceAMT, bondSCD, cpnLST, dc30)

# 価格 >> 利回り計算
PrToYLD   = bondOBJ.bondYield(cP(97.0), dc30, cmpdCMP, freqSA)  # cP(97.0)に修正
print(f'価格97.0の利回り : PrToYLD = {PrToYLD:6%}')

価格97.0の利回り : PrToYLD = 1.418713%


In [2]:
# 2. 利回り >> 価格計算
# bondSCDのメソッド    
prevCpnDT = bondSCD.previousDate(settleDT)
accruDays = dc30.dayCount(prevCpnDT, settleDT)

# bondOBJのメソッド
accruAMT = bondOBJ.accruedAmount()
cleanPRC = bondOBJ.cleanPrice(PrToYLD, dc30, cmpdCMP, freqSA)
dirtyPRC = bondOBJ.dirtyPrice(PrToYLD, dc30, cmpdCMP, freqSA)
# 保存
dfPRC = pd.DataFrame([ 
            ['受渡日', settleDT.to_date()], ['前回利払日',prevCpnDT.to_date()],
            ['経過日数', accruDays],        ['経過利息',accruAMT],
            ['---- ---- ----', ''],
            ['クリーン価格', cleanPRC],     ['利含み価格',dirtyPRC], 
            ['---- ---- ----', ''] ],       columns=['計算結果',''])   ; dfPRC

,計算結果,
0,受渡日,2022-08-23
1,前回利払日,2022-07-28
2,経過日数,25
3,経過利息,0.025694
4,---- ---- ----,
5,クリーン価格,97.000001
6,利含み価格,97.025695
7,---- ---- ----,


In [3]:
# 3. リスク指標の算出
# InterestRateオブジェクトの作成
iRateOBJ = ql.InterestRate(PrToYLD, dc30, cmpdCMP, freqSA)

# BondFunctionによる算出
MacDUR = ql.BondFunctions.duration(       bondOBJ, iRateOBJ, ql.Duration.Macaulay)
ModDUR = ql.BondFunctions.duration(       bondOBJ, iRateOBJ, ql.Duration.Modified)
BPV    = ql.BondFunctions.basisPointValue(bondOBJ, iRateOBJ)
convX  = ql.BondFunctions.convexity(      bondOBJ, iRateOBJ)
# 保存
dfRSK = pd.DataFrame([ 
            ['マコーレーDur.', MacDUR],  ['修正Dur.', ModDUR],
            ['BPV', BPV],              ['Convexity', convX],
            ['---- ---- ----', ''] ],  columns=['計算結果',''])   ; dfRSK

,計算結果,
0,マコーレーDur.,2.916496
1,修正Dur.,2.895954
2,BPV,-0.028098
3,Convexity,9.84958
4,---- ---- ----,


In [4]:
# BPVの手計算(hand calculation)
Pup1bp = bondOBJ.dirtyPrice(PrToYLD+0.0001, dc30, cmpdCMP, freqSA)
Pdw1bp = bondOBJ.dirtyPrice(PrToYLD-0.0001, dc30, cmpdCMP, freqSA)
hcBPV  = (Pup1bp - Pdw1bp)/2 ; print(f'BPV(hc): {hcBPV:.6f}円')

BPV(hc): -0.028098円


In [5]:
# 修正Durationの手計算
hcMoDur = -BPV*100 / dirtyPRC  ; 
print(f'modDur(hc): {hcMoDur:.6%}', end=',  ' )
# Convexity
hcCnvx = ((Pup1bp-dirtyPRC) - (dirtyPRC-Pdw1bp))*10000/dirtyPRC
print(f'Convexity(hc): {hcCnvx:.6%}')

modDur(hc): 2.895949%,  Convexity(hc): 0.098496%


In [6]:
# 1%金利上昇時の価格
Pup100bp = bondOBJ.dirtyPrice(PrToYLD+0.01, dc30, cmpdCMP, freqSA)

deltA = -ModDUR/100*dirtyPRC ; gammA = convX/10000*dirtyPRC
hcPRC = dirtyPRC + deltA + 0.5*gammA
print(f'1%上昇時価格: {Pup100bp:.6f}, デルタ+1/2ガンマの推定: {hcPRC:.6f}')


1%上昇時価格: 94.263045, デルタ+1/2ガンマの推定: 94.263659


* テキスト p121 図4.6と表示を多少変えた(style.format(fmtFUT)を使用)
* 利含み価格を (dfBND.amount *dfBND.DF).sum() で計算 (図4.3に一致)

In [7]:
# クーポン
dfCPN = pd.DataFrame({
    'payDate':    cpn.date(),          # no ISO
    'coupon':     cpn.rate(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'amount':     cpn.amount(),
    } for cpn in map(ql.as_coupon, bondOBJ.cashflows()) if cpn is not None )
# 起算日, 元本
dfEFF = pd.DataFrame([{'payDate': bondOBJ.startDate()}], columns=dfCPN.columns)
dfPRN = pd.DataFrame({'payDate': cf.date(), 'amount':cf.amount()} for cf,cpn 
             in zip(bondOBJ.cashflows(),map(ql.as_coupon, bondOBJ.cashflows())) 
                                                               if cpn is None )
dfBND = pd.concat([dfEFF, dfCPN, dfPRN], ignore_index=True )
# ディスカウントファクター
psDF  = [1.0                  for dt in dfBND.payDate if dt <= settleDT] #past   DF
fuDF  = [iRateOBJ.discountFactor(settleDT, dt)
                              for dt in dfBND.payDate if settleDT < dt ] #future DF
dfBND = pd.concat([dfBND, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)  
dfBND.payDate  =  dfBND.payDate.map(lambda x: x.ISO())            # ISOフォーマットへ
display( dfBND.style.format(fmtFUT) )
print(f'(hc) 利含み価格:{(dfBND.amount *dfBND.DF).sum():.8f}')

,payDate,coupon,accruStart,accruEnd,amount,DF
0,2022-07-28,nan%,nan,nan,nan,1.000000
1,2023-01-30,0.3700%,2022-07-28,2023-01-28,0.1850,0.993854
2,2023-07-28,0.3700%,2023-01-28,2023-07-28,0.1850,0.986931
3,2024-01-29,0.3700%,2023-07-28,2024-01-28,0.1850,0.979941
4,2024-07-29,0.3700%,2024-01-28,2024-07-28,0.1850,0.973038
5,2025-01-28,0.3700%,2024-07-28,2025-01-28,0.1850,0.966223
6,2025-07-28,0.3700%,2025-01-28,2025-07-28,0.1850,0.959417
7,2025-07-28,nan%,nan,nan,100.0000,0.959417


(hc) 利含み価格:97.02569537


In [8]:
# マコーレーデュレーション 手計算
yrFRC = [dc30.yearFraction(settleDT, iDT(xx)) for xx in dfBND.payDate][1:]
np.sum(yrFRC*dfBND.amount[1:]*dfBND.DF[1:])/dirtyPRC

np.float64(2.916496208263682)

In [9]:
# 2025年以降のキャッシュフローの抽出
dfBND.payDate = dfBND.payDate.map(lambda x: iDT(x)) #ISO日付をDateクラスへ
dfBND = dfBND[dfBND.payDate >= jDT(2025,1,1)]
dfBND = dfBND.reset_index(drop=True)           ; dfBND

,payDate,coupon,accruStart,accruEnd,amount,DF
0,"January 28th, 2025",0.0037,2024-07-28,2025-01-28,0.185,0.966223
1,"July 28th, 2025",0.0037,2025-01-28,2025-07-28,0.185,0.959417
2,"July 28th, 2025",NaN,NaN,NaN,100.000,0.959417


In [10]:
# 債券の評価 - **Bonds - MitHCC 0.37 7/28/25**

import xlwings as xw ; from myABBR import * #; import myUtil as mu
@xw.func
@xw.arg('trdDATA', ndim=1)
@xw.arg('bondDTL', ndim=1)
@xw.arg('bondCONV', ndim=1)
@xw.ret(index=False, header=True, expand='table')

def bondCALC(trdDATA, bondDTL, bondCONV):
    # 0. パラメーター編集と初期設定
    tradeDT, settDS, mktYLD      = trdDATA       ;  settDS = int(settDS)
    effDT, matDT, faceAMT, cpnLST = bondDTL
    calBD, cmpdBD, dcBD, freqBD, cpnCNV, matCNV, dtGenBD, EoMBD       =\
                                                 [eval(ii) for ii in bondCONV]
    
    tradeDT,         effDT,       matDT,    cpnLST               =\
    dDT(tradeDT), dDT(effDT), dDT(matDT),  [cpnLST] 
    settleDT = calBD.advance(tradeDT, settDS, DD) ; setEvDT(tradeDT)

    # 1. スケジュールオブジェクトと債券オブジェクトの作成
    bondSCD = ql.Schedule(effDT, matDT, pD(freqBD), calBD, cpnCNV, matCNV, dtGenBD, EoMBD)
    bondOBJ = ql.FixedRateBond(settDS, faceAMT, bondSCD, cpnLST, dcBD)

    # 2. 利回り >> 価格計算
    # bondSCDのメソッド    
    preCpnDT = bondSCD.previousDate(settleDT)
    accruDS = dcBD.dayCount(preCpnDT, settleDT)

    # bondOBJのメソッド
    accruAMT = bondOBJ.accruedAmount()
    cleanPRC = bondOBJ.cleanPrice(mktYLD, dcBD, cmpdBD, freqBD)
    dirtyPRC = bondOBJ.dirtyPrice(mktYLD, dcBD, cmpdBD, freqBD)
    # 保存
    dfPRC = pd.DataFrame([ 
                ['受渡日', settleDT.to_date()], ['前回利払日',preCpnDT.to_date()],
                ['経過日数', accruDS],          ['経過利息',accruAMT],
                ['---- ---- ----', ''],
                ['クリーン価格', cleanPRC],     ['利含み価格',dirtyPRC], 
                ['---- ---- ----', ''] ],  columns=['計算結果',''])   ; dfPRC
        
    # 3. リスク指標の算出
    # InterestRateオブジェクトの作成
    iRateOBJ = ql.InterestRate(mktYLD, dcBD, cmpdBD, freqBD)

    # BondFunctionによる算出
    MacDUR = ql.BondFunctions.duration(       bondOBJ, iRateOBJ, ql.Duration.Macaulay)
    ModDUR = ql.BondFunctions.duration(       bondOBJ, iRateOBJ, ql.Duration.Modified)
    BPV    = ql.BondFunctions.basisPointValue(bondOBJ, iRateOBJ)
    convX  = ql.BondFunctions.convexity(      bondOBJ, iRateOBJ)
    # 保存
    dfRSK = pd.DataFrame([['マコーレーDur.', MacDUR], ['修正Dur.', ModDUR],
                           ['BPV', BPV],               ['Convexity', convX], ], 
                columns=['計算結果',''])
    return pd.concat([dfPRC, dfRSK], ignore_index=True)

In [11]:
# TONAカーブ作成とその日付リスト
crvDATA = [('depo','1d',-0.009),('swap','1m',-0.01807),('swap','6m',-0.01043),
           ('swap','12m',0.0125),('swap','18m',0.03125),('swap','2y',0.04875),
           ('swap','3y',0.07375), ('swap','5y',0.11854),('swap','7y',0.19146)]

tonaIX, tnCrvOBJ, tnCrvHDL, tnParRT = mu.makeTonaCurve(crvDATA)
tnCrvDates = [dt.serialNumber() for dt,_ in tnCrvOBJ.nodes()[1:]]

# TONA補間レート、iSpread
linearIP  = ql.LinearInterpolation(tnCrvDates, tnParRT)
intPOrate = linearIP(matDT.serialNumber(), allowExtrapolation=True)
print(f'TONA補間レート({matDT.ISO()}): {intPOrate:.4%}, ',
         f'債券利回り: {PrToYLD:.4%},  Iスプレッド: {PrToYLD-intPOrate:.4%}')

TONA補間レート(2025-07-28): 0.0718%,  債券利回り: 1.4187%,  Iスプレッド: 1.3469%


In [12]:
# 利回り1.4187%の利含み価格
ffCrvOBJ,ffCrvHDL = mu.ffTSH(settleDT, 0.01418713, dc30, cmpdCMP, freqSA)
ffENG = ql.DiscountingBondEngine(ffCrvHDL)
bondOBJ.setPricingEngine(ffENG)  
print(f'利回り1.4187%での利含み価格 {bondOBJ.NPV():.6f}')

利回り1.4187%での利含み価格 97.025696


In [13]:
# TONAカーブでの債券価格
tnENG = ql.DiscountingBondEngine(tnCrvHDL); bondOBJ.setPricingEngine(tnENG)
tnDiscPRC = bondOBJ.NPV()            #ASスプレッドで使用する為tnDiscPRCへ保存
print(f'TONAカーブでの価格(tnDiscPRC): {tnDiscPRC:.6f}')

TONAカーブでの価格(tnDiscPRC): 100.897045


In [14]:
# clean価格=97.0のzSpread
zSpread = ql.BondFunctions.zSpread(bondOBJ,cP(97.0),tnCrvOBJ,dc30,cmpdCMP,freqSA)   # cP(97.0)に修正
print(f'クリーン価格 97.0のZスプレッド: {zSpread:.4%}')

クリーン価格 97.0のZスプレッド: 1.3459%


In [15]:
# TONAカーブを1.3459%シフトした時の債券価格
sprdCvOBJ = ql.ZeroSpreadedTermStructure(
                tnCrvHDL, mu.sqHDL(1.3459/100), cmpdCMP, freqA, dcA365)
sprdCvHDL = ql.YieldTermStructureHandle(sprdCvOBJ)
sprdENG   = ql.DiscountingBondEngine(sprdCvHDL)
bondOBJ.setPricingEngine(sprdENG)
print(f'tonaカーブ+Zスプレッドでの利含み価格: {bondOBJ.NPV():.6f}')

tonaカーブ+Zスプレッドでの利含み価格: 97.025690


In [16]:
# TONAカーブ+Zスプレッドでの価格算出
dfBND = mu.bondCashFlow(bondOBJ, yts=sprdCvHDL)
fmtSCF.update(amount='{:,.4f}')
display(dfBND.style.format(fmtSCF))
print(f'Zスプレッドの価格(手計算): {(dfBND.amount * dfBND.DF).sum():.6f}')

,payDate,coupon,accruStart,accruEnd,amount,DF
0,2023-01-30,0.370000%,2022-07-28,2023-01-28,0.1850,0.99405822
1,2023-07-28,0.370000%,2023-01-28,2023-07-28,0.1850,0.98741962
2,2024-01-29,0.370000%,2023-07-28,2024-01-28,0.1850,0.98044049
3,2024-07-29,0.370000%,2024-01-28,2024-07-28,0.1850,0.97345996
4,2025-01-28,0.370000%,2024-07-28,2025-01-28,0.1850,0.96637951
5,2025-07-28,0.370000%,2025-01-28,2025-07-28,0.1850,0.95941373
6,2025-07-28,nan%,nan,nan,100.0000,0.95941373


Zスプレッドの価格(手計算): 97.025690


In [17]:
# ASスプレッド手計算
dfBND = mu.bondCashFlow(bondOBJ, yts=tnCrvOBJ)[:-1] ; display(dfBND)
yrFRC = [dcA365.yearFraction(settleDT,iDT(xx)) for xx in dfBND.payDate]
tnrFL  = np.diff([0]+yrFRC)                              # 利払い日のテナー
tnAnn = (dfBND.DF * tnrFL).sum()                         # TONAアニュイティ
# ASスプレッド
print(f'TONAアニュイティ:{tnAnn:.6f}, '
      f'ASスプレッド:{(tnDiscPRC - dirtyPRC)/tnAnn:.4f}')

,payDate,coupon,accruStart,accruEnd,amount,DF
0,2023-01-30,0.0037,2022-07-28,2023-01-28,0.185,1.000048
1,2023-07-28,0.0037,2023-01-28,2023-07-28,0.185,0.999902
2,2024-01-29,0.0037,2023-07-28,2024-01-28,0.185,0.999581
3,2024-07-29,0.0037,2024-01-28,2024-07-28,0.185,0.999096
4,2025-01-28,0.0037,2024-07-28,2025-01-28,0.185,0.998491
5,2025-07-28,0.0037,2025-01-28,2025-07-28,0.185,0.997880


TONAアニュイティ:2.929009, ASスプレッド:1.3217


In [18]:
# ASスプレッド手計算
dfBND = mu.bondCashFlow(bondOBJ, yts=tnCrvOBJ)[:-1] ; display(dfBND)
yrFRC = [dcA365.yearFraction(settleDT,iDT(xx)) for xx in dfBND.payDate]
tnrFL  = np.diff([0]+yrFRC)                              # 利払い日のテナー
tnAnn = (dfBND.DF * tnrFL).sum()                         # TONAアニュイティ
# ASスプレッド
print(f'TONAアニュイティ:{tnAnn:.6f}, '
      f'ASスプレッド:{(tnDiscPRC - 90.0256)/tnAnn:.4f}')

,payDate,coupon,accruStart,accruEnd,amount,DF
0,2023-01-30,0.0037,2022-07-28,2023-01-28,0.185,1.000048
1,2023-07-28,0.0037,2023-01-28,2023-07-28,0.185,0.999902
2,2024-01-29,0.0037,2023-07-28,2024-01-28,0.185,0.999581
3,2024-07-29,0.0037,2024-01-28,2024-07-28,0.185,0.999096
4,2025-01-28,0.0037,2024-07-28,2025-01-28,0.185,0.998491
5,2025-07-28,0.0037,2025-01-28,2025-07-28,0.185,0.997880


TONAアニュイティ:2.929009, ASスプレッド:3.7116


- 下セル(図4.18)の**AssetSwap**クラスはver1.39以降、RFR指数の使用で仕様変更
  - 2行目で定義した空の`fltSCH変数`を3行目で次のように修正  
    `fltSCH = ql.Schedule(settleDT, matDT, pdFreqA, calJP, unADJ,unADJ, dtGENb, EoMf)`  
    (この引数は図4.2の9行目のbondSCDを多少修正したもの)
  - 本来 **空のfltSCH変数**の指定はアセットスワップされる債券のスケジュールを参照していた (`fltSCH=bondSCD`に同じ)

In [19]:
# AS用パラメータ
cleanPRC,  crdSPRD, isPar,  payFix                        = \
97.0,     1.3/100,  True,   True
fltSCH = ql.Schedule(settleDT, matDT, pdFreqA, calJP, unADJ,unADJ, dtGENb, EoMf)
# ASオブジェクトとASエンジン
aswOBJ = ql.AssetSwap(payFix, bondOBJ, cleanPRC, 
                                    tonaIX, crdSPRD, fltSCH, dcA365, isPar)
aswENG = ql.DiscountingSwapEngine(tnCrvHDL); aswOBJ.setPricingEngine(aswENG)

print(f'クリーン価格{cleanPRC:.2f}のASスプレッド:{aswOBJ.fairSpread():.4%}\n', 
      f'スプレッド{crdSPRD:.4%}のクリーン価格: {aswOBJ.fairCleanPrice():.4f}')

クリーン価格97.00のASスプレッド:1.3221%
 スプレッド1.3000%のクリーン価格: 97.0646
